# ANISE

ANISE is a modern rewrite of NAIF SPICE, written in Rust and providing interfaces to other languages including Python.

Evidently, this tutorial applies to the Python usage of ANISE.

## Goal
By the end of this tutorial, you should know how to build a data frame containing the Azimuth, Elevation, and Range data from a location from any body fixed object (like an Earth ground station) and another other object stored in the almanac.

Let's start by installing ANISE: `pip install anise`

## Loading the latest orientation and planetary data

In this tutorial, we're using the latest Earth orientation parameters from JPL, and the latest planetary ephemerides, constants, and high precision Moon rotation parameters. These can be downloaded automatically by ANISE using the MetaAlmanac class (refer to tutorial #02 for details).

In [1]:
from anise import MetaAlmanac

almanac = MetaAlmanac.latest()

almanac

Almanac: #SPK = 1	#BPC = 2	PlanetaryData with 54 ID mappings and 0 name mappings	EulerParameterData with 3 ID mappings and 3 name mappings (@0x5638ac745070)

Superb! We've got two BPCs loaded (the latest Earth and the long-term Moon high precision rotation), the DE440s planetary ephemerides, and planetary constants for 49 objects in the solar system.

## Defining a ground station

Let's define a ground station in Madrid, Spain (for example). We start by importing the Orbit structure, and figure out how it could be initialized for a ground asset.

In [2]:
from anise.astro import Orbit
Orbit.from_latlongalt?

Signature:
Orbit.from_latlongalt(
    latitude_deg,
    longitude_deg,
    height_km,
    angular_velocity,
    epoch,
    frame,
)
Docstring:
Creates a new Orbit from the latitude (φ), longitude (λ) and height (in km) with respect to the frame's ellipsoid given the angular velocity.

**Units:** degrees, degrees, km, rad/s
NOTE: This computation differs from the spherical coordinates because we consider the flattening of body.
Reference: G. Xu and Y. Xu, "GPS", DOI 10.1007/978-3-662-50367-6_2, 2016
Type:      builtin_function_or_method

In [3]:
# Define the location of the asset
# Build the Madrid DSN gound station
latitude_deg = 40.427_222
longitude_deg = 4.250_556
height_km = 0.834_939

# As noted in the documentation, this call also requires the mean angular velocity of the reference frame.
# Source: G. Xu and Y. Xu, "GPS", DOI 10.1007/978-3-662-50367-6_2, 2016 (confirmed by https://hpiers.obspm.fr/eop-pc/models/constants.html)
MEAN_EARTH_ANGULAR_VELOCITY_DEG_S = 0.004178079012116429

## Defining the frame and epoch

In [4]:
from anise.astro.constants import Frames
from anise.time import *

# Grab the high precision Earth rotation frame (ITRF93)
itrf93 = almanac.frame_info(Frames.EARTH_ITRF93)
itrf93

Earth ITRF93 (μ = 398600.435436096 km^3/s^2, eq. radius = 6378.1366 km, polar radius = 6356.7519 km, f = 0.0033528131084554717) (@0x7f64028c6e80)

We now have everything to initialize the ground asset, apart from the epoch. So let's define a time series to compute the AER of the (center) of the Moon throughout a month. 

In [5]:
start = Epoch("2023-01-01 20:00:00")
end = start + Unit.Day*30
step = Unit.Minute*1

time_series = TimeSeries(start, end, step, inclusive=True)
time_series

TimeSeries { start: 2023-01-01T20:00:00 UTC, duration: Duration { centuries: 0, nanoseconds: 2592000000000000 }, step: Duration { centuries: 0, nanoseconds: 60000000000 }, cur: 0, incl: true } @ 0x7f64032ba1a0

## Generate the AER data
Now, we just need to iterate over this time series, storing the AER result in an array for each computation.

Note that the nomenclature used in the AER computation is "receiver" and "transmitter", which is the typical convention for orbit determination, a capability enabled by the Nyx astrodynamics package, a superset of ANISE.

In [6]:
almanac.azimuth_elevation_range_sez?

Signature: almanac.azimuth_elevation_range_sez(rx, tx)
Docstring:
Computes the azimuth (in degrees), elevation (in degrees), and range (in kilometers) of the
receiver state (`rx`) seen from the transmitter state (`tx`), once converted into the SEZ frame of the transmitter.

# Algorithm
1. Compute the SEZ (South East Zenith) frame of the transmitter.
2. Rotate the receiver position vector into the transmitter SEZ frame.
3. Rotate the transmitter position vector into that same SEZ frame.
4. Compute the range as the norm of the difference between these two position vectors.
5. Compute the elevation, and ensure it is between +/- 180 degrees.
6. Compute the azimuth with a quadrant check, and ensure it is between 0 and 360 degrees.
Type:      builtin_function_or_method

In [7]:
# Import the aberration correction because we want to correct for the light time.
from anise import Aberration
# Checking the doc is always helpful!
almanac.transform?

Signature: almanac.transform(target_frame, observer_frame, epoch, ab_corr=None)
Docstring:
Returns the Cartesian state needed to transform the `from_frame` to the `to_frame`.

# SPICE Compatibility
This function is the SPICE equivalent of spkezr: `spkezr(TARGET_ID, EPOCH_TDB_S, ORIENTATION_ID, ABERRATION, OBSERVER_ID)`
In ANISE, the TARGET_ID and ORIENTATION are provided in the first argument (TARGET_FRAME), as that frame includes BOTH
the target ID and the orientation of that target. The EPOCH_TDB_S is the epoch in the TDB time system, which is computed
in ANISE using Hifitime. THe ABERRATION is computed by providing the optional Aberration flag. Finally, the OBSERVER
argument is replaced by OBSERVER_FRAME: if the OBSERVER_FRAME argument has the same orientation as the TARGET_FRAME, then this call
will return exactly the same data as the spkerz SPICE call.

# Note
The units will be those of the underlying ephemeris data (typically km and km/s)
Type:      builtin_function_or_method

In [8]:
aer_data = []

for epoch in time_series:
    # Build the observer ("transmitter") at this epoch
    tx = Orbit.from_latlongalt(latitude_deg, longitude_deg, height_km, MEAN_EARTH_ANGULAR_VELOCITY_DEG_S, epoch, itrf93)
    # Grab the state of the Moon at this epoch ("receiver")
    rx = almanac.transform(Frames.MOON_J2000, Frames.EARTH_J2000, epoch, Aberration("LT+S"))
    # NOTE: The rx state here is in the Earth J2000 frame, but that's OK because the Almanac will compute the translations needed.
    this_aer = almanac.azimuth_elevation_range_sez(rx, tx)
    aer_data += [this_aer]

## Plotting the data

Let's build a Polars dataframe with the AER data and plot it using `hvplot` (the recommended tool by Polars). We're using Polars here because it's dozens of times faster than Pandas, and we've got ~43k rows in our data set, so we might as well filter through it quickly.

Let's install Polars (with the plotting option) and hvplot.

In [11]:
%pip install "polars[plot]" hvplot


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Now, let's build a data frame. As of January 2024, we need some custom handling of the epochs in Hifitime to make them compatible with Python's lower precision datetime object, but this will be fixed in [this hifitime issue](https://github.com/nyx-space/hifitime/issues/185).

In [13]:
import polars as pl
from datetime import datetime

def hifitime_to_datetime(e: Epoch) -> datetime:
    return datetime.fromisoformat(str(e).replace(" UTC", "")[:23])

# Build the arrays of data
epochs = []
elevations = []
azimuths = []
ranges = []
range_rates = []

for aer in aer_data:
    epochs += [hifitime_to_datetime(aer.epoch)]
    elevations += [aer.elevation_deg]
    azimuths += [aer.azimuth_deg]
    ranges += [aer.range_km]
    range_rates += [aer.range_rate_km_s]

# Build the data frame
df = pl.DataFrame(
    {
        'epochs': epochs,
        'elevation_deg': elevations,
        'azimuth_deg': azimuths,
        'range_km': ranges,
        'range_rate_km_s': range_rates
    }
)

df.describe()

describe,epochs,elevation_deg,azimuth_deg,range_km,range_rate_km_s
str,str,f64,f64,f64,f64
"""count""","""43201""",43201.0,43201.0,43201.0,43201.0
"""null_count""","""0""",0.0,0.0,0.0,0.0
"""mean""",null,1.920361,179.943419,385666.088874,0.009381
"""std""",null,36.571063,98.443342,16472.238106,13.798014
"""min""","""2023-01-01 20:…",-77.166677,0.005663,354094.280434,-20.694402
"""25%""",null,-25.144,92.887676,370637.2144,-13.742085
"""50%""",null,1.205067,180.070913,390142.67677,-0.0035
"""75%""",null,29.304787,266.940744,399983.931307,13.763225
"""max""","""2023-01-31 20:…",76.760723,359.975421,409264.590342,20.698013


In [14]:
import hvplot.polars
from bokeh.models.formatters import DatetimeTickFormatter

formatter = DatetimeTickFormatter(days='%d/%m') # Make the world a better place

df.hvplot(x="epochs", y="elevation_deg", xformatter=formatter, title="Elevation of the Moon throughout Jan 2023 from Madrid, Spain")

:Curve   [epochs]   (elevation_deg)

Great! We've plotted the elevation of the Moon over the month of January 2023. Let's now plot the azimuth angle only when the elevation is greater than 5 degrees.

In [15]:
elevation_gt_5 = df.filter(pl.col('elevation_deg') > 5)
elevation_gt_5.hvplot(x="epochs", y="azimuth_deg", xformatter=formatter, title="Azimuth of the Moon when above horizon from Madrid, Spain")

:Curve   [epochs]   (azimuth_deg)

In [16]:
df.hvplot(x="epochs", y="range_km", xformatter=formatter, title="Range of the Moon from Madrid, Spain throughout Jan 2023")

:Curve   [epochs]   (range_km)

In [17]:
df.hvplot(x="epochs", y="range_rate_km_s", xformatter=formatter, title="Range rate of the Moon from Madrid, Spain throughout Jan 2023")

:Curve   [epochs]   (range_rate_km_s)

## Exercises

### Learning objective

1. Become more familiar with TimeSeries
2. Become more familiar with building assets on the surface of the Moon and Earth
3. Learn how to convert between units using Hifitime's `Unit` structure, and plotting those results.

### 1. Range and light-time from Earth asset to Pluto until 2045

+ Build a time series from today (`Epoch.system_now()`) until a date of your choosing in 2045, with a daily time step.
+ Build the AER data for the whole time series.
+ Using the speed of light constant defined in `anise.astro.constants` in the `UsualConstants` class, compute the light time delay from the range data.
+ Plot the range over time and plot the light time in hours over time (by multiplying the light time in seconds by `Unit.Second` and calling `to_unit(Unit.Hour)` on the result).

### 2. Elevation from a lunar asset to an Earth asset

+ Using the mean Lunar angular velocity constant defined in `anise.astro.constants` in the `UsualConstants` class, compute the elevation of the Earth as seen from Shackleton Crater on the Lunar South pole.

Hints: You will need to build the Moon frame with either data from the PCK08 Planetary Constants or from the high precision Moon orientation parameters. Use `almanac.describe()` to see a pretty printed list of the loaded data to know which frame to build. Or you may use the MOON_PA frame from the planetary constants.